<a href="https://colab.research.google.com/github/ctrivino1/YLearn/blob/main/YLearn_global_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/ctrivino1/YLearn.git

In [ ]:
# data
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import pandas as pd
from ylearn.estimator_model.double_ml import DoubleML
from xgboost import XGBRegressor
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

housing = fetch_california_housing(as_frame=True)
data = housing.frame
outcome = housing.target_names[0]
data[outcome] = housing.target
data.head()

In [ ]:
outcome

In [ ]:
from ylearn.estimator_model import double_ml
from ylearn import Why
# identify
why = Why()
r=why.identify(data, outcome,treatment_count_limit=2)

print('identified treatment:',r[0])
print('identified adjustment:',r[1])
print('identified covariate:',r[2])
print('identified instrument:',r[3])

In [ ]:
# Assuming 'your_outcome_column' is the outcome variable name
x_model = XGBRegressor()  # You can choose a different model if needed
y_model = XGBRegressor()  # You can choose a different model if needed

double_ml_model = double_ml.DoubleML(
    x_model=x_model,
    y_model=y_model,
    cf_fold=10,  # You can adjust the cross-fitting folds
    random_state=2022,
)

train_size = int(0.8 * len(data))
train_data = data[:train_size]
val_data = data[train_size:]
# Fit the DoubleML model (this is just an example, you need to specify the model and instruments)
double_ml_model.fit(data,outcome,treatment=r[0],covariate=r[2])

estimated_ate = double_ml_model.estimate(val_data,quantity='ATE')
#print(f"Treatment{double_ml_model.treatment}")
#print(f"Treatment ATE: {estimated_ate}")
y_intercept = double_ml_model.yx_model.intercept_
#print(f"Y Intercept: {y_intercept}")
coefficents = double_ml_model.yx_model.coef_
#print(f"coefficients: {coefficents}")

In [ ]:
double_ml_model.yx_model.coef_

In [ ]:
# convert to array just extracts the covariate data and turns it into a numpy array
print(double_ml.global_v_convert2array)
print(data[['AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']])

In [ ]:
# both wv and v will be equal unless a variabes w is introduced, but I haven't ran into a situation like this yet
print(double_ml.global_wv_data )
print("space")
print(double_ml.global_v_convert2array)

print("")
are_equal = np.array_equal(double_ml.global_wv_data, double_ml.global_v_convert2array)
print("Are double_ml.global_wv_data and double_ml.global_v_convert2array equal?", are_equal)

In [ ]:
print(double_ml.cross_fit_y_target) # med house val

In [ ]:
# x.shape:  (20640, 2)
#y.shape:  (20640, 1)
x_hat = double_ml.global_x_hat_dict ["paras"][0].reshape((20640, 2))
print(pd.DataFrame(x_hat))
print(pd.DataFrame(double_ml.global_x_convert2array))

In [ ]:
# looking at how x_prime is calculated

In [ ]:
def nd_kron(x, y):
    assert x.shape[0] == y.shape[0]
    fn = np.vectorize(np.kron, signature="(n),(m)->(k)")
    kron_prod = fn(x, y)

    return kron_prod

In [ ]:
x_diff = double_ml.global_x_convert2array - x_hat
v = np.hstack([np.ones((double_ml.global_v_convert2array .shape[0], 1)), double_ml.global_v_convert2array ])
print(v[0])
x_diff[0]

In [ ]:
assert (20640, 2)[0] == (20640, 1)[0]
fn = np.vectorize(np.kron, signature="(n),(m)->(k)")
print(fn)
kron_prod = fn(x_diff[0], v[0])
print(kron_prod)


In [ ]:
F = double_ml_model.yx_model.coef_
v = double_ml.global_v_convert2array
result = np.dot(F, v.T)
result